In [52]:
import numpy as np
import pandas as pd
import constants
from functions import *
from constants import *
from importlib import reload

In [53]:
consants = reload(constants)
functions = reload(functions)

In [34]:
Po = np.zeros([ndt+1, ndx, ndy])
Pw = np.zeros([ndt+1, ndx, ndy])
Po[0] = Poi

In [35]:
Qo = np.zeros([ndt+1, ndx, ndy])
Qw = np.zeros([ndt+1, ndx, ndy])
Qo[0] = qo
Qw[0] = qw

In [36]:
S_wr = 0.2
S_or = 0.1
S_wi = 0.2

In [37]:
S_w = np.zeros([ndt+1, ndx, ndy])
S_o = np.zeros([ndt+1, ndx, ndy])
S_w[0] = S_wi
S_o[0] = 1 - S_wi

In [55]:
PHI = np.zeros([ndt, ndx, ndy])
Pc = np.zeros([ndt, ndx, ndy])
dPc = np.zeros([ndt, ndx, ndy])
dBo = np.zeros([ndt, ndx, ndy])
dBw = np.zeros([ndt, ndx, ndy])
ToW = np.zeros([ndt, ndx, ndy])
ToE = np.zeros([ndt, ndx, ndy])
ToN = np.zeros([ndt, ndx, ndy])
ToS = np.zeros([ndt, ndx, ndy])
TwW = np.zeros([ndt, ndx, ndy])
TwE = np.zeros([ndt, ndx, ndy])
TwN = np.zeros([ndt, ndx, ndy])
TwS = np.zeros([ndt, ndx, ndy])
Cpoo = np.zeros([ndt, ndx, ndy])
Cpow = np.zeros([ndt, ndx, ndy])
Cswo = np.zeros([ndt, ndx, ndy])
Csww = np.zeros([ndt, ndx, ndy])
Alpha = np.zeros([ndt, ndx, ndy])
Viso = np.zeros([ndt, ndx, ndy])
Visw = np.zeros([ndt, ndx, ndy])
Bo = np.zeros([ndt, ndx, ndy])
Bw = np.zeros([ndt, ndx, ndy])
t = 0

In [56]:
Pc[0], _ = capillary_pressure(S_wr, S_or, S_w[0], ndx, ndy)

Pw[0] = Po[0] - Pc[0]

In [57]:
Vis_o, Vis_w = viscosity(Po[0], Pw[0], ndx, ndy)

In [69]:
Bo, Bw, dBo, dBw = FVF(Po, Pw, Bo_0, Bw_0, Co, Cw, P_0, ndx, ndy, t)
Kr_o, Kr_w = relative_permeability(S_wr, S_or, S_w[0], ndx, ndy)
lambda_o, lambda_w = mobility(BCW, BCE, BCN, BCS, Kr_o[0], Kr_w[0], Vis_o[0], Vis_w[0], Bo[0], Bw[0], Po[0], Pw[0], ndx, ndy)

ValueError: setting an array element with a sequence.

In [67]:
(S_w[0][3][7] - S_wr) / (1-S_or-S_wr)

0.0

In [72]:
0.9 * np.power(1-S_w[0], 2)

array([[0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576,
        0.576],
       [0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576,
        0.576],
       [0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576,
        0.576],
       [0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576,
        0.576],
       [0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576,
        0.576],
       [0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576,
        0.576],
       [0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576,
        0.576],
       [0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576,
        0.576],
       [0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576,
        0.576],
       [0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576, 0.576,
        0.576]])

In [30]:
Jo = 2 * 3.14 * 1.127 * np.multiply(Kx, dz, lambda_o[0]) / (np.log(r_eq/rw_o) + S_o[0])
Jw = 2 * 3.14 * 1.127 * Kx * dz * (Bo[0] / Bw[0] * lambda_o[0] + lambda_w[0]) / (np.log(r_eq/rw_w) + S_w[0])
Jow = 2 * 3.14 * 1.127 * np.multiply(Kx, dz, lambda_w[0]) / (np.log(r_eq/rw_o) + S_o[0])

NameError: name 'lambda_o' is not defined

In [31]:
if IBo == 1 and IBw == 1:
    BHPo = Po[0] + Qo[0] / Jo
    BHPw = Pw[0] + Qw[0] / Jw
    Qwo = Qo[0] * lambda_w[0] / lambda_o[0]